In [1]:
import json
import pandas as pd
import os
import re
import glob  # determine if this is needed
import numpy as np

In [2]:
# Display all columns in the pd data frame
pd.set_option('display.max_columns', None)

### Stage 1. Read the files, extract the AODV messages and make data frames for each node

In [3]:
# Set default AODV parameters
# Most of these will not be required so can delete them. 
# Keep for now incase I will build routing tables.
NODE_TRAVERSAL_TIME = 10e-3 # 10 milliseconds
NET_DIAMETER = 20
NET_TRAVERSAL_TIME = 2 * NODE_TRAVERSAL_TIME * NET_DIAMETER
PATH_DISCOVERY_TIME = 2 * NET_TRAVERSAL_TIME
ACTIVE_ROUTE_TIMEOUT = 2 * NET_TRAVERSAL_TIME
DELETE_PERIOD = 5 * NET_TRAVERSAL_TIME
RREQ_RETRIES = 3
RREQ_RATELIMIT = 10
RERR_RATELIMIT = 10
HELLO_INTERVAL = 1 # 1000 milliseconds
ALLOWED_HELLO_LOSS = 2
NEXT_HOP_WAIT = 2 * NODE_TRAVERSAL_TIME
BLACKLIST_TIMEOUT = RREQ_RETRIES * NET_TRAVERSAL_TIME # Double check RREQ_RETRIES, there was atypo in the document.

In [4]:
# Get the current working directory
os.getcwd()

'C:\\Users\\alang\\ITS_CP'

In [5]:
print("Ensure that the directory only contains the necessary.json files")
#directory_path = input("Set Working Directory. Include ./: ")
directory_path = 'Datasets\DS2\Test_1' 

Ensure that the directory only contains the necessary.json files


In [7]:
# Verify that the directory path contains the right files only.
print(os.listdir(directory_path))
print()
print(directory_path)

['Node0_DS2_Test1.json', 'Node1_DS2_Test1.json', 'Node2_DS2_Test1.json', 'Node3_DS2_Test1.json', 'Node4_DS2_Test1.json', 'Node5_DS2_Test1.json']

Datasets\DS2\Test_1


In [8]:
# read all the Node files into a list.
# use os.listdir() to get a list of all files in the directory
all_files = os.listdir(directory_path)

# use a list comprehension to filter the .json files
node_files = [file for file in all_files if file.endswith('.json')]

node_files

['Node0_DS2_Test1.json',
 'Node1_DS2_Test1.json',
 'Node2_DS2_Test1.json',
 'Node3_DS2_Test1.json',
 'Node4_DS2_Test1.json',
 'Node5_DS2_Test1.json']

In [9]:
# This list contains the black hole node IDs.
black_hole_list = ['49', '45']

# Concatenate the ip address to the black hole node ID.
for i, node in enumerate(black_hole_list):
    black_hole_list[i] = '10.1.1.' + node

print(black_hole_list)

['10.1.1.49', '10.1.1.45']


#### Stage 1 functions

In [10]:
# It is assumed that the RREQ is received and not sent.
def get_rreq_info(aodv):
    sn = aodv['aodv.orig_ip']
    dn = aodv['aodv.dest_ip']
    hc = aodv['aodv.hopcount']
    ssn = aodv['aodv.orig_seqno']
    dsn = aodv['aodv.dest_seqno']
    bid = aodv['aodv.rreq_id']
    return sn, dn, hc, ssn, dsn, bid

In [11]:
# It is assumed that the RREP is received and not sent.
def get_rrep_info(aodv):
    sn = aodv['aodv.orig_ip']
    dn = aodv['aodv.dest_ip']
    hc = aodv['aodv.hopcount']
    dsn = aodv['aodv.dest_seqno']
    return sn, dn, hc, dsn

In [12]:
def get_rrer_info(aodv):
    dn = aodv['Unreachable Destinations']['aodv.unreach_dest_ip']
    hc = aodv['aodv.destcount']
    dsn = aodv['Unreachable Destinations']['aodv.dest_seqno']
    return dn, hc, dsn

In [13]:
# Max Sequence Number allowed in AODV is 2**32 - 1 = 4294967295
def calc_seq_inc(seq_rrep, seq_rreq):
    max_seq_num = 2**32 -1
    diff = seq_rrep - seq_rreq
    # If the maximum sequence number has been exceeded, calculate the difference.
    if diff < -1e7:
        diff = max_seq_num + seq_rrep - seq_rreq
    return diff

In [14]:
# Make a list of the node IDs
node_id_list = []
for item in node_files:
    match = re.search(r'\d+', item)
    if match:
        node_id_list.append(match.group())

print(node_id_list)

['0', '1', '2', '3', '4', '5']


### Stage 1 for loop
#### This will loop through all the .json files and convert them into data frames of AODV messages as seen by each node.

In [15]:
stage_2_list = []

In [16]:
for index in range(len(node_files)):
    # Find the number of the current node and store as a string
    crt_node = '10.1.1.' + node_id_list[index]
    print(f'Current Node: {crt_node}')
    type_4 = 0
    
    # Read the current json file and load it.
    file_path = os.path.join(directory_path, node_files[index])
    j = open(file_path, 'r')
    j1 = json.load(j)
    # Close the JSON file
    j.close()
    
    # Create an empty data frame
    df_aodv = pd.DataFrame(columns = ['Msg_Index', 'frame_time', 'frame_time_relative', 'This_Node', 'Nbr_Node', 
                                      'Next_Hop', 'TTL', 'AODV_Msg', 'Source_Node', 'Destination_Node', 'Hop_Count', 
                                      'Source_Seq_Num', 'Dest_Seq_Num', 'Broadcast_ID'])
    
    # This for loop will iterate through all the AODV messaging lines in the JSON file and convert them into a 
    # data frame that can be further processed in Stage 2.
    for idx in range(0,len(j1)):
        if 'aodv' in j1[idx]['_source']['layers'].keys():
            frame_time = j1[idx]['_source']['layers']['frame']['frame.time_epoch']
            frame_time_rel = j1[idx]['_source']['layers']['frame']['frame.time_relative']
            # Double check the next two final keys as there are multiple options. All appear to have the same values so far.
            ip_src = j1[idx]['_source']['layers']['ip']['ip.src']
            ip_dest = j1[idx]['_source']['layers']['ip']['ip.dst']
            ttl = j1[idx]['_source']['layers']['ip']['ip.ttl']
            # Possibly remove the udp timers.
            udp_time_rel = j1[idx]['_source']['layers']['udp']['Timestamps']['udp.time_relative']
            udp_time_delta = j1[idx]['_source']['layers']['udp']['Timestamps']['udp.time_delta']
            aodv_tp = j1[idx]['_source']['layers']['aodv']['aodv.type']
            if aodv_tp == '1':
                aodv_type = 'RREQ'
                q_j1 = get_rreq_info(j1[idx]['_source']['layers']['aodv'])
                source_node = q_j1[0]
                dest_node = q_j1[1]
                hop_count = q_j1[2]
                s_seq_num = q_j1[3]
                d_seq_num = q_j1[4]
                bc_id = q_j1[5]

            elif aodv_tp == '2':
                aodv_type = 'RREP'
                p_j1 = get_rrep_info(j1[idx]['_source']['layers']['aodv'])
                source_node = p_j1[0]
                dest_node = p_j1[1]
                hop_count = p_j1[2]
                d_seq_num = p_j1[3]
                s_seq_num = None
                bc_id = None

            elif aodv_tp == '3':
                aodv_type = 'RERR'
                r_j1 = get_rrer_info(j1[idx]['_source']['layers']['aodv'])
                source_node = None
                dest_node = r_j1[0]
                hop_count = r_j1[1]
                d_seq_num = r_j1[2]
                s_seq_num = None
                bc_id = None

            elif aodv_tp == '4':
                # This is not relevant and causes issues. It will be removed later along with the messages
                # that respond to it.
                #print('"aodv.type": "4"')
                aodv_type = 'RREP-ACK'
                source_node = None
                dest_node = None
                hop_count = 0
                d_seq_num = 0
                s_seq_num = 0
                bc_id = 0
                type_4 += 1
            
            else:
                print("Error")

            new_row = {'Msg_Index': idx+1, 'frame_time': frame_time, 'frame_time_relative': frame_time_rel,
                       'This_Node': crt_node, 'Nbr_Node': ip_src, 'TTL': ttl, 'Next_Hop': ip_dest, 'AODV_Msg': aodv_type,
                       'Source_Node': source_node, 'Destination_Node': dest_node, 'Hop_Count': hop_count,
                       'Source_Seq_Num': s_seq_num, 'Dest_Seq_Num': d_seq_num, 'Broadcast_ID': bc_id}

            df_aodv = df_aodv.append(new_row, ignore_index=True)
        
    # Convert various columns from string to float or integer.
    df_aodv['Msg_Index'] = df_aodv['Msg_Index'].astype(int)
    df_aodv['frame_time'] = df_aodv['frame_time'].astype(float)
    df_aodv['frame_time_relative'] = df_aodv['frame_time_relative'].astype(float)
    df_aodv['TTL'] = df_aodv['TTL'].astype(int)
    df_aodv['Hop_Count'] = df_aodv['Hop_Count'].astype(int)
    df_aodv['Source_Seq_Num'] = pd.to_numeric(df_aodv['Source_Seq_Num'], errors='coerce').astype('Int64')
    df_aodv['Dest_Seq_Num'] = df_aodv['Dest_Seq_Num'].astype(int)
    df_aodv['Broadcast_ID'] = pd.to_numeric(df_aodv['Broadcast_ID'], errors='coerce').astype('Int64')

    """
    # Write the Stage 1 .csv files
    file_name = f'Node_{node_id_list[index]}_Stage_1.csv'
    dir_path = directory_path + "\\"
    file_path = dir_path + file_name
    df_aodv.to_csv(file_path)
    """

    # Store the data frame in a list for Stage 2
    stage_2_list.append(df_aodv)
    print(f'{type_4} RREP-ACK messages found')
    print()

Current Node: 10.1.1.0
138 RREP-ACK messages found

Current Node: 10.1.1.1
147 RREP-ACK messages found

Current Node: 10.1.1.2
158 RREP-ACK messages found

Current Node: 10.1.1.3
78 RREP-ACK messages found

Current Node: 10.1.1.4
58 RREP-ACK messages found

Current Node: 10.1.1.5
95 RREP-ACK messages found



In [17]:
stage_2_list[0].head(15)

,Msg_Index,frame_time,frame_time_relative,This_Node,Nbr_Node,Next_Hop,TTL,AODV_Msg,Source_Node,Destination_Node,Hop_Count,Source_Seq_Num,Dest_Seq_Num,Broadcast_ID
0,1,0.011304,0.000000,10.1.1.0,10.1.1.45,10.1.1.255,1,RREP,10.1.1.45,10.1.1.45,0,<NA>,0,<NA>
1,2,0.027304,0.016000,10.1.1.0,10.1.1.2,10.1.1.255,1,RREP,10.1.1.2,10.1.1.2,0,<NA>,0,<NA>
2,3,0.034050,0.022746,10.1.1.0,10.1.1.1,10.1.1.255,1,RREP,10.1.1.1,10.1.1.1,0,<NA>,0,<NA>
3,4,1.004304,0.993000,10.1.1.0,10.1.1.45,10.1.1.255,1,RREP,10.1.1.45,10.1.1.45,0,<NA>,0,<NA>
4,5,1.027304,1.016000,10.1.1.0,10.1.1.2,10.1.1.255,1,RREP,10.1.1.2,10.1.1.2,0,<NA>,0,<NA>
5,6,1.030050,1.018746,10.1.1.0,10.1.1.1,10.1.1.255,1,RREP,10.1.1.1,10.1.1.1,0,<NA>,0,<NA>
6,7,2.005304,1.994000,10.1.1.0,10.1.1.45,10.1.1.255,1,RREP,10.1.1.45,10.1.1.45,0,<NA>,0,<NA>
7,8,2.029304,2.018000,10.1.1.0,10.1.1.2,10.1.1.255,1,RREP,10.1.1.2,10.1.1.2,0,<NA>,0,<NA>
8,9,2.036050,2.024746,10.1.1.0,10.1.1.1,10.1.1.255,1,RREP,10.1.1.1,10.1.1.1,0,<NA>,0,<NA>
9,10,3.007304,2.996000,10.1.1.0,10.1.1.45,10.1.1.255,1,RREP,10.1.1.45,10.1.1.45,0,<NA>,0,<NA>


### Stage 2. Extract certain information from the messaging and modify the data frames

### Work on the next data frame before creating the dataset
**Here I need to do the following:**

1. Determine if the RREP is a genuine RREP or a 'Hello'
2. If it is a genuine RREP, Determine which is the RREQ message it is responding too. 
<br>
<span style="color: red;">This needs to be modified once the RREP in the .pcap files are correct.</span>
3. If genuine RREP, Calculate the time difference between the RREP and RREQ and create a field, RREP response time.
   Note if the Sequence Number is approaching the limit 2^32 or 4,294,967,296 This will have to be taken into account
4. Make a feature of Dest_Seq_Num_Increment
5. Make a feature of Orig_Seq_Num_Increment - This should be 0 so likely remove.
6. Create another field of RREP response time per num hops = RREP response time / Hop_Count
7. Make a Feature RREP Hopcount <= 1
8. Make a feature RREP hopcount >=2 ??

<br>

**To do later**
1. Add data packets to the data frame

<span style="color: red;">**Maybe add code to ignore warnings**</span>

**For the dataset**
1. Make a feature Number of NBRs - I will be lazy at first and do the entire number of nbrs during the whole sim, not just for each time period. This may lead to inaccuracies in the modelling.

In [19]:
# Set some parameters for creating the table
# The increment below is considered to be too high and indicative of a BHN
# This value needs to be determined after unning simulations. 20 is an example
HIGH_DEST_SEQ_NUM_INC = 20
# Assume maximum response time for RREP to be received after and RREQ is sent
MAX_RESP_TIME = 6

### Stage 2 for loop
#### This will loop through all the data frames created in Stage 1 and add some new features for each row.

In [20]:
stage_3_list = []

In [21]:
for df_idx in range(len(stage_2_list)):
    df_mod = stage_2_list[df_idx].copy()
    # Maybe remove these features
    df_mod['This_Node_Is_Dest'] = ""
    df_mod['This_Node_Is_Orig'] = ""
    # Create new columns
    df_mod['Hello'] = ""
    df_mod['Nbr_Is_Orig'] = ""
    df_mod['Nbr_Is_Dest'] = ""
    df_mod['RREQ_Msg_Idx'] = ""
    df_mod['RREP_Resp_Time'] = ""
    df_mod['RREP_Resp_Time_Per_Hop'] = ""
    df_mod['Hop_Cnt_Over_1'] = ""
    df_mod['Dest_Seq_Num_Increment'] = ""
    df_mod['Orig_Seq_Num_Increment'] = ""
    # Maybe remove these Features
    df_mod['Tagged_For_Del'] = ""
    
    type_4 = 0
    no_rreq = 0
    
    # Rows that are affected by the RREP-ACK message will be removed.
    delete_list = []
    print(f'Working on Node {node_id_list[df_idx]} *******************************************')
    
    # Modify the data frame to create more features.
    for idx in range(len(df_mod)):
        # RERR Messages do not contain useful information and cause problems in the .pcap file
        # They will be removed.
        if df_mod['AODV_Msg'][idx] == 'RREP-ACK':
            delete_list.append(df_mod['Msg_Index'][idx])
            type_4 += 1
            df_mod['Tagged_For_Del'][idx] = 'RREP-ACK'
        
        # Check if This Node is source or destination
        if df_mod['This_Node'][idx] == df_mod['Source_Node'][idx]:
            df_mod['This_Node_Is_Orig'][idx] = 'Yes'           
        else:
            df_mod['This_Node_Is_Orig'][idx] = 'No'
            
        if df_mod['This_Node'][idx] == df_mod['Destination_Node'][idx]:
            df_mod['This_Node_Is_Dest'][idx] = 'Yes'
        else:
            df_mod['This_Node_Is_Dest'][idx] = 'No'
         
        # Extract information from the RREQ messages
        if df_mod['AODV_Msg'][idx] == 'RREQ':
            if df_mod['Nbr_Node'][idx] == df_mod['Source_Node'][idx]:
                df_mod['Nbr_Is_Orig'][idx] = True
            else:
                df_mod['Nbr_Is_Orig'][idx] = False

        # Extract information from the RREP messages
        # Determine if the RREP message is a "Hello" broadcast.
        if df_mod['AODV_Msg'][idx] == 'RREP':
            if (df_mod['Next_Hop'][idx].endswith('.255') and 
               (df_mod['Source_Node'][idx] == df_mod['Destination_Node'][idx])):
                # Condition met for Hello, label as such
                df_mod['Hello'][idx] = True

            # If RREP is not a "Hello" broadcast.
            else:
                df_mod['Hello'][idx] = False
                # Determine if the neighbor was the destination node of the RREP
                if ((df_mod['Source_Node'][idx] != df_mod['Destination_Node'][idx]) and
                    (df_mod['Nbr_Node'][idx] == df_mod['Destination_Node'][idx])):
                    # The neighbour is the destination of the RREP message
                    df_mod['Nbr_Is_Dest'][idx] = True

                else:
                    # Neighbor is not the destination node
                    df_mod['Nbr_Is_Dest'][idx] = False

                candidate_idx = None
                idx2 = idx # This starts from the current RREP but will not crash in the first iteration.
                
                # Only RREQ messages within the max response time will be checked.
                # This while loop will determine the 'candidate' or correct RREQ message to which the RREP responded.
                # Modify code and add comments here on how this is done once the corrected .pcap files have been obtained.
                while (df_mod['frame_time_relative'][idx] - df_mod['frame_time_relative'][idx2]) < MAX_RESP_TIME:
                    #print(idx2, df_mod['frame_time_relative'][idx] - df_mod['frame_time_relative'][idx2])
                    if df_mod['AODV_Msg'][idx2] == 'RREQ':
                        #print(idx2, df_mod['Msg_Index'][idx2], df_mod['frame_time_relative'][idx2], df_mod['AODV_Msg'][idx2], 
                              #df_mod['Next_Hop'][idx2],
                              #df_mod['Source_Node'][idx2], df_mod['Destination_Node'][idx2])
                        # Determine if RREQ is valid for RREP
                        # It will be valid if the Source_Node and Destination_Node are the same and RREP Hop_Count < RREQ TTL
                        # This code will be modified once the RREP in the .pcap files are correct!!!
                        if ((df_mod['Source_Node'][idx2] == df_mod['Source_Node'][idx]) and 
                            (df_mod['Destination_Node'][idx2] == df_mod['Destination_Node'][idx]) and 
                            (df_mod['TTL'][idx2] >= df_mod['Hop_Count'][idx])):
                            if candidate_idx == None:
                                #print()
                                #print(f"Candidate, idx2: {idx2}, TTL: {df_mod['TTL'][idx2]}, RREP Hop_Count: {df_mod['Hop_Count'][idx]}")
                                candidate_idx = idx2
                            else:
                                if df_mod['frame_time_relative'][idx2] <= df_mod['frame_time_relative'][candidate_idx]:
                                    #print()
                                    #print(f"Candidate, idx2: {idx2}, TTL: {df_mod['TTL'][idx2]}, 
                                    #RREP Hop_Count: {df_mod['Hop_Count'][idx]}")
                                    candidate_idx = idx2
                                #else:
                                    #print()
                                    #print(f"Not A Candidate, idx2: {idx2}, TTL: {df_mod['TTL'][idx2]},
                                    #RREP Hop_Count: {df_mod['Hop_Count'][id
                                    
                    idx2 -= 1
                    if idx2 < 0: # If the index is less than 0, break the loop
                        break
    
                # If the correct RREQ message was found pertaining to the RREP
                if candidate_idx != None:
                    #print(f"final candidate: {candidate_idx}, TTL: {df_mod['TTL'][candidate_idx]}")
                    # Update the required columns
                    df_mod['RREQ_Msg_Idx'][candidate_idx] = idx
                    df_mod['RREQ_Msg_Idx'][idx] = candidate_idx
                    df_mod['RREP_Resp_Time'][idx] = \
                        df_mod['frame_time_relative'][idx] - df_mod['frame_time_relative'][candidate_idx]
                    # + 1 added to the demoniator as the hop count should be incrememented when recieved by the node. 
                    # This will also avoid a demoniator of 0.
                    df_mod['RREP_Resp_Time_Per_Hop'][idx] = \
                        df_mod['RREP_Resp_Time'][idx] / (df_mod['Hop_Count'][idx] + 1)
                    df_mod['Dest_Seq_Num_Increment'][idx] = \
                        calc_seq_inc(df_mod['Dest_Seq_Num'][idx], df_mod['Dest_Seq_Num'][candidate_idx])
                    # df_mod['Orig_Seq_Num_Increment'][idx] = calc_seq_inc(df_mod['Orig_Seq_Num'][idx], df_mod['Orig_Seq_Num'][candidate_idx])

                # If the RREQ was not found
                else:
                    # print(f'No RREQ found for RREP Index {idx}. This is likely due to an error caused by an RREP-ACK message')
                    no_rreq += 1
                    delete_list.append(df_mod['Msg_Index'][idx])
                    df_mod['Tagged_For_Del'][idx] = 'No matching RREQ'

                # Determine if the number of hop counts is greater than 1 if Nbr is not the destination
                if  (df_mod['Nbr_Is_Dest'][idx] == False):
                    if (df_mod['Hop_Count'][idx] > 1):
                        df_mod['Hop_Cnt_Over_1'][idx] = True
                    else:
                        df_mod['Hop_Cnt_Over_1'][idx] = False
                    
    
    # Write the Stage 2 .csv files
    file_name = f'Node_{node_id_list[df_idx]}_Stage_2.csv'
    dir_path = directory_path + "\\"
    file_path = dir_path + file_name
    df_mod.to_csv(file_path)
    
    # The rows will be deleted for Stage 3 but will be in the Stage 2 files.
    print(f'{type_4} RREP-ACK messages. and {no_rreq} RREPs with no matching RREQ. They will be deleted')
    # filter the rows to be deleted based on the values in column Msg_Index
    rows_to_delete = df_mod[df_mod['Msg_Index'].isin(delete_list)].index
    # drop the rows from the dataframe
    df_mod = df_mod.drop(index=rows_to_delete)
    # Reset the indices
    df_mod = df_mod.reset_index()

    
    # Store the data frame in a list for Stage 3
    stage_3_list.append(df_mod)
                
    print()

Working on Node 0 *******************************************


C:\Users\alang\anaconda3\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\alang\anaconda3\lib\site-packages\ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\alang\anaconda3\lib\site-packages\ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\alang\anaconda3\lib\site-packages\ipykernel_launcher.py:49: SettingWithCopyWarning: 
A valu

138 RREP-ACK messages. and 632 RREPs with no matching RREQ. They will be deleted

Working on Node 1 *******************************************


C:\Users\alang\anaconda3\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\alang\anaconda3\lib\site-packages\ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


147 RREP-ACK messages. and 704 RREPs with no matching RREQ. They will be deleted

Working on Node 2 *******************************************
158 RREP-ACK messages. and 850 RREPs with no matching RREQ. They will be deleted

Working on Node 3 *******************************************
78 RREP-ACK messages. and 426 RREPs with no matching RREQ. They will be deleted

Working on Node 4 *******************************************
58 RREP-ACK messages. and 615 RREPs with no matching RREQ. They will be deleted

Working on Node 5 *******************************************
95 RREP-ACK messages. and 512 RREPs with no matching RREQ. They will be deleted



### Stage 3. Create individual datasets and merge them into one dataset.

In [22]:
node_dataset_list = []

In [23]:
for df_idx in range (len(stage_3_list)):
    print(f'Working on Node {node_id_list[df_idx]} ***************************************************************')
    df_mod = stage_3_list[df_idx].copy()
    # Remove all rows where the Current Node is the same as the Neighbour Node
    #df_mod = df_mod[df_mod['This_Node'] != df_mod['Nbr_Node']]
    # Reset the indices
    df_mod = df_mod.reset_index()
    # Create an empty dataset
    ds_temp = pd.DataFrame(columns = ['Node', 'Nbr_Node', 'Nbr_Count', 'Hello_Cnt', 'AODV_Msg_Nbr_Cnt', 'RREQs_Sent_To_Nbr',
                                      'RREQs_From_Nbr', 'Nbr_Is_Orig_Cnt', 'Nbr_Never_Orig', 'Nbr_Is_Dest_Cnt',
                                      'Nbr_Never_Dest', 'All_RREPs_Rcvd_This_Node', 'RREPs_From_Nbr', 'RREP_Resp_Pct',
                                      'Pct_Of_All_RREPs', 'Hop_Cnt_Over_1_Cnt', 'Hop_Cnt_Over_1_Pct', 
                                      'High_Dest_Seq_Num_Inc_Cnt', 'High_Dest_Seq_Num_Inc_Pct', 'Avg_Resp_Dly',
                                      'Avg_Resp_Dly_Per_Hop', 'RERRs_From_Nbr', 'RERRs_From_Nbr_Pct', 'Pct_of_All_Nbrs',
                                      'RREP_To_Nbrs_Ratio', 'Black_Hole_Node'])
    
    # Create Empty lists
    # Concatenation of Source Node and Broadcast ID
    sn_bc_id_list = []
    
    # Create the Individual dataset
    node = df_mod['This_Node'][0]
    All_RREPs_Rcvd_This_Node = 0
    RREQs_Sent_To_Nbr = 0
    for idx in range(0,len(df_mod)):
        if (df_mod['Nbr_Node'][idx] not in ds_temp['Nbr_Node'].values):
            # Create a new row
            new_row = {'Node': node, 'Nbr_Count': 0, 'Nbr_Node': df_mod['Nbr_Node'][idx], 'Hello_Cnt': 0, 
                       'AODV_Msg_Nbr_Cnt': 0, 'RREQs_Sent_To_Nbr': 0, 'RREQs_From_Nbr': 0, 'Nbr_Is_Orig_Cnt': 0,
                       'Nbr_Never_Orig': False, 'Nbr_Is_Dest_Cnt': 0, 'Nbr_Never_Dest': False, 
                       'All_RREPs_Rcvd_This_Node': 0, 'RREPs_From_Nbr': 0, 'RREP_Resp_Pct': np.nan, 
                       'Pct_Of_All_RREPs': np.nan, 'Hop_Cnt_Over_1_Cnt': 0, 
                       'Hop_Cnt_Over_1_Pct': np.nan, 'High_Dest_Seq_Num_Inc_Cnt': 0, 'High_Dest_Seq_Num_Inc_Pct': np.nan, 
                       'Avg_Resp_Dly': 0, 'Avg_Resp_Dly_Per_Hop': 0, 'RERRs_From_Nbr': 0, 'RERRs_From_Nbr_Pct': np.nan,
                       'Pct_of_All_Nbrs': np.nan, 'RREP_To_Nbrs_Ratio': np.nan, 'Black_Hole_Node': False}

            # Add the new row
            ds_temp = ds_temp.append(new_row, ignore_index=True)

        # Add the appropriate values
        # If message is "Hello Broadcast" then increment counter
        if df_mod['Hello'][idx]:
            ds_temp.loc[ds_temp['Nbr_Node'] == df_mod['Nbr_Node'][idx], 'Hello_Cnt'] += True

        # check if a message in not a Hello and incremenet the neighbour's counter
        if (df_mod['AODV_Msg'][idx] in ('RREQ', 'RREP', 'RERR')) and (df_mod['Hello'][idx] != True):
            ds_temp.loc[ds_temp['Nbr_Node'] == df_mod['Nbr_Node'][idx], 'AODV_Msg_Nbr_Cnt'] += 1

        # Check if TTL > 1
        if ((df_mod['AODV_Msg'][idx] == 'RREQ') and (df_mod['TTL'][idx] > 1) and 
            (df_mod['This_Node'][idx] != df_mod['Nbr_Node'][idx])):
            sn_bc_id = df_mod['Source_Node'][idx] + '_' + str(df_mod['Broadcast_ID'][idx])
            if sn_bc_id not in sn_bc_id_list:
                sn_bc_id_list.append(sn_bc_id)
                RREQs_Sent_To_Nbr += 1

        # Set RREQ Features
        if df_mod['AODV_Msg'][idx] == 'RREQ':
            ds_temp.loc[ds_temp['Nbr_Node'] == df_mod['Nbr_Node'][idx], 'RREQs_From_Nbr'] += 1

            if df_mod['Source_Node'][idx] == df_mod['Nbr_Node'][idx]:
                ds_temp.loc[ds_temp['Nbr_Node'] == df_mod['Nbr_Node'][idx], 'Nbr_Is_Orig_Cnt'] += 1

            if df_mod['Destination_Node'][idx] == df_mod['Nbr_Node'][idx]:
                ds_temp.loc[ds_temp['Nbr_Node'] == df_mod['Nbr_Node'][idx], 'Nbr_Is_Dest_Cnt'] += 1

        # Set RREP Features
        if (df_mod['AODV_Msg'][idx] == 'RREP') and (df_mod['Hello'][idx] == False):
            All_RREPs_Rcvd_This_Node += 1
            ds_temp.loc[ds_temp['Nbr_Node'] == df_mod['Nbr_Node'][idx], 'RREPs_From_Nbr'] += 1
            # Note: Below the totals delays are being accumulated. They will be averaged by the RREPs_From_Nbr after the loop.
            ds_temp.loc[ds_temp['Nbr_Node'] == df_mod['Nbr_Node'][idx], 'Avg_Resp_Dly'] += df_mod['RREP_Resp_Time'][idx]
            ds_temp.loc[ds_temp['Nbr_Node'] == df_mod['Nbr_Node'][idx], \
                    'Avg_Resp_Dly_Per_Hop'] += df_mod['RREP_Resp_Time_Per_Hop'][idx]

            if df_mod['Hop_Cnt_Over_1'][idx]:
                ds_temp.loc[ds_temp['Nbr_Node'] == df_mod['Nbr_Node'][idx], 'Hop_Cnt_Over_1_Cnt'] += 1
            if df_mod['Dest_Seq_Num_Increment'][idx] > HIGH_DEST_SEQ_NUM_INC:
                ds_temp.loc[ds_temp['Nbr_Node'] == df_mod['Nbr_Node'][idx], 'High_Dest_Seq_Num_Inc_Cnt'] += 1

        # Set RERR Features
        if df_mod['AODV_Msg'][idx] == 'RERR':
            ds_temp.loc[ds_temp['Nbr_Node'] == df_mod['Nbr_Node'][idx], 'RERRs_From_Nbr'] += 1
            
    
    # Add totals to the appropriate columns
    ds_temp['All_RREPs_Rcvd_This_Node'] = All_RREPs_Rcvd_This_Node
    ds_temp['RREQs_Sent_To_Nbr'] = RREQs_Sent_To_Nbr
    # Set appropriate data types
    ds_temp['Nbr_Count'] = ds_temp['Nbr_Count'].astype(int)
    ds_temp['Hello_Cnt'] = ds_temp['Hello_Cnt'].astype(int)
    ds_temp['AODV_Msg_Nbr_Cnt'] = ds_temp['AODV_Msg_Nbr_Cnt'].astype(int)
    ds_temp['RREQs_Sent_To_Nbr'] = ds_temp['RREQs_Sent_To_Nbr'].astype(int)
    ds_temp['RREQs_From_Nbr'] = ds_temp['RREQs_From_Nbr'].astype(int)
    ds_temp['Nbr_Is_Orig_Cnt'] = ds_temp['Nbr_Is_Orig_Cnt'].astype(int)
    ds_temp['Nbr_Never_Orig'] = ds_temp['Nbr_Never_Orig'].astype(bool)
    ds_temp['Nbr_Is_Dest_Cnt'] = ds_temp['Nbr_Is_Dest_Cnt'].astype(int)
    ds_temp['Nbr_Never_Dest'] = ds_temp['Nbr_Never_Dest'].astype(bool)
    ds_temp['All_RREPs_Rcvd_This_Node'] = ds_temp['All_RREPs_Rcvd_This_Node'].astype(int)
    ds_temp['RREPs_From_Nbr'] = ds_temp['RREPs_From_Nbr'].astype(int)
    ds_temp['RREP_Resp_Pct'] = ds_temp['RREP_Resp_Pct'].astype(float)
   # ds_temp['RREP_Resp_Pct_2'] = ds_temp['RREP_Resp_Pct_2'].astype(float)
    ds_temp['Pct_Of_All_RREPs'] = ds_temp['Pct_Of_All_RREPs'].astype(float)
    ds_temp['Hop_Cnt_Over_1_Cnt'] = ds_temp['Hop_Cnt_Over_1_Cnt'].astype(int)
    ds_temp['Hop_Cnt_Over_1_Pct'] = ds_temp['Hop_Cnt_Over_1_Pct'].astype(float)
    ds_temp['High_Dest_Seq_Num_Inc_Cnt'] = ds_temp['High_Dest_Seq_Num_Inc_Cnt'].astype(int)
    ds_temp['High_Dest_Seq_Num_Inc_Pct'] = ds_temp['High_Dest_Seq_Num_Inc_Pct'].astype(float)
    ds_temp['Avg_Resp_Dly'] = ds_temp['Avg_Resp_Dly'].astype(float)
    ds_temp['Avg_Resp_Dly_Per_Hop'] = ds_temp['Avg_Resp_Dly_Per_Hop'].astype(float)
    ds_temp['RERRs_From_Nbr'] = ds_temp['RERRs_From_Nbr'].astype(int)
    ds_temp['RERRs_From_Nbr_Pct'] = ds_temp['RERRs_From_Nbr_Pct'].astype(float)
    ds_temp['Pct_of_All_Nbrs'] = ds_temp['Pct_of_All_Nbrs'].astype(float)
    ds_temp['RREP_To_Nbrs_Ratio'] = ds_temp['RREP_To_Nbrs_Ratio'].astype(float)
    ds_temp['Black_Hole_Node'] = ds_temp['Black_Hole_Node'].astype(bool)
    
    # Change the remaining values
    # Change the target variable to True if the Neighbour Node is in the black hole list
    ds_temp.loc[ds_temp['Nbr_Node'].isin(black_hole_list), 'Black_Hole_Node'] = True
    # Number of Neighbors for this Node
    ds_temp['Nbr_Count'] = len(ds_temp['Nbr_Node'].values)
    # Nbr never originates a RREQ
    ds_temp['Nbr_Never_Orig'] = (ds_temp['Nbr_Is_Orig_Cnt'] == 0)
    # Nbr is never the destination of an RREQ
    ds_temp['Nbr_Never_Dest'] = (ds_temp['Nbr_Is_Dest_Cnt'] == 0)

    # Calculate to percentage of RREP Responses from RREQs.
    # There are 2 counters. Decide which one I prefer.
    ds_temp.loc[ds_temp['RREQs_Sent_To_Nbr'] != 0, 'RREP_Resp_Pct'] = \
        round((100 * ds_temp['RREPs_From_Nbr'] / ds_temp['RREQs_Sent_To_Nbr']).astype(float),2)

    ds_temp.loc[ds_temp['All_RREPs_Rcvd_This_Node'] != 0, 'Pct_Of_All_RREPs'] = \
        round((100 * ds_temp['RREPs_From_Nbr'] / ds_temp['All_RREPs_Rcvd_This_Node']).astype(float),2)

    ds_temp.loc[ds_temp['RREPs_From_Nbr'] != 0, 'Hop_Cnt_Over_1_Pct'] = \
        round((100 * ds_temp['Hop_Cnt_Over_1_Cnt'] / ds_temp['RREPs_From_Nbr']).astype(float),2)

    ds_temp.loc[ds_temp['RREPs_From_Nbr'] != 0, 'High_Dest_Seq_Num_Inc_Pct'] = \
        round((100 * ds_temp['High_Dest_Seq_Num_Inc_Cnt'] / ds_temp['RREPs_From_Nbr']).astype(float),2)

    ds_temp.loc[ds_temp['RREPs_From_Nbr'] != 0, 'RERRs_From_Nbr_Pct'] = \
        round((100 * ds_temp['RERRs_From_Nbr'] / ds_temp['RREPs_From_Nbr']).astype(float),2)

    ds_temp.loc[ds_temp['Nbr_Count'] != 0, 'Pct_of_All_Nbrs'] = \
        round((100 * 1 / ds_temp['Nbr_Count']).astype(float),2)

    ds_temp.loc[ds_temp['Pct_of_All_Nbrs'] != 0, 'RREP_To_Nbrs_Ratio'] = \
        round((100 * ds_temp['Pct_Of_All_RREPs'] / ds_temp['Pct_of_All_Nbrs']).astype(float),2)

    ds_temp['Avg_Resp_Dly'] = np.where(ds_temp['RREPs_From_Nbr'] != 0, \
           (ds_temp['Avg_Resp_Dly'] / ds_temp['RREPs_From_Nbr']), np.nan)

    ds_temp['Avg_Resp_Dly_Per_Hop'] = np.where(ds_temp['RREPs_From_Nbr'] != 0, \
           (ds_temp['Avg_Resp_Dly_Per_Hop'] / ds_temp['RREPs_From_Nbr']), np.nan)
    
    # Append this data set to the list
    node_dataset_list.append(ds_temp)
    print(f'Completed Working on Node {node_id_list[df_idx]} ***************************************************************')
    print()

Working on Node 0 ***************************************************************
Completed Working on Node 0 ***************************************************************

Working on Node 1 ***************************************************************
Completed Working on Node 1 ***************************************************************

Working on Node 2 ***************************************************************
Completed Working on Node 2 ***************************************************************

Working on Node 3 ***************************************************************
Completed Working on Node 3 ***************************************************************

Working on Node 4 ***************************************************************
Completed Working on Node 4 ***************************************************************

Working on Node 5 ***************************************************************
Completed Working on Node 5 ***************

In [24]:
node_dataset_list[0]

,Node,Nbr_Node,Nbr_Count,Hello_Cnt,AODV_Msg_Nbr_Cnt,RREQs_Sent_To_Nbr,RREQs_From_Nbr,Nbr_Is_Orig_Cnt,Nbr_Never_Orig,Nbr_Is_Dest_Cnt,Nbr_Never_Dest,All_RREPs_Rcvd_This_Node,RREPs_From_Nbr,RREP_Resp_Pct,Pct_Of_All_RREPs,Hop_Cnt_Over_1_Cnt,Hop_Cnt_Over_1_Pct,High_Dest_Seq_Num_Inc_Cnt,High_Dest_Seq_Num_Inc_Pct,Avg_Resp_Dly,Avg_Resp_Dly_Per_Hop,RERRs_From_Nbr,RERRs_From_Nbr_Pct,Pct_of_All_Nbrs,RREP_To_Nbrs_Ratio,Black_Hole_Node
0,10.1.1.0,10.1.1.45,44,25,0,181,0,0,True,0,True,621,0,0.00,0.00,0,NaN,0,NaN,NaN,NaN,0,NaN,2.27,0.00,True
1,10.1.1.0,10.1.1.2,44,56,127,181,56,0,True,0,True,621,54,29.83,8.70,41,75.93,0,0.0,3.525608,0.946357,17,31.48,2.27,383.26,False
2,10.1.1.0,10.1.1.1,44,151,262,181,149,5,False,0,True,621,69,38.12,11.11,31,44.93,0,0.0,1.826252,0.649025,44,63.77,2.27,489.43,False
3,10.1.1.0,10.1.1.3,44,42,158,181,63,3,False,0,True,621,67,37.02,10.79,22,32.84,0,0.0,1.357565,0.553015,28,41.79,2.27,475.33,False
4,10.1.1.0,10.1.1.31,44,24,49,181,24,0,True,0,True,621,15,8.29,2.42,8,53.33,0,0.0,0.502173,0.199722,10,66.67,2.27,106.61,False
5,10.1.1.0,10.1.1.12,44,7,0,181,0,0,True,0,True,621,0,0.00,0.00,0,NaN,0,NaN,NaN,NaN,0,NaN,2.27,0.00,False
6,10.1.1.0,10.1.1.41,44,8,0,181,0,0,True,0,True,621,0,0.00,0.00,0,NaN,0,NaN,NaN,NaN,0,NaN,2.27,0.00,False
7,10.1.1.0,10.1.1.50,44,8,0,181,0,0,True,0,True,621,0,0.00,0.00,0,NaN,0,NaN,NaN,NaN,0,NaN,2.27,0.00,False
8,10.1.1.0,10.1.1.27,44,41,0,181,0,0,True,0,True,621,0,0.00,0.00,0,NaN,0,NaN,NaN,NaN,0,NaN,2.27,0.00,False
9,10.1.1.0,10.1.1.18,44,30,20,181,14,10,False,0,True,621,3,1.66,0.48,1,33.33,0,0.0,0.956295,0.407157,3,100.00,2.27,21.15,False


In [25]:
# Create the final dataset
dataset_merged = pd.concat(node_dataset_list)
# Reset the indices
dataset_merged = dataset_merged.rename_axis('Index').reset_index()
dataset_merged['Index'] = range(1, len(dataset_merged) + 1)

In [26]:
dataset_merged

,Index,Node,Nbr_Node,Nbr_Count,Hello_Cnt,AODV_Msg_Nbr_Cnt,RREQs_Sent_To_Nbr,RREQs_From_Nbr,Nbr_Is_Orig_Cnt,Nbr_Never_Orig,Nbr_Is_Dest_Cnt,Nbr_Never_Dest,All_RREPs_Rcvd_This_Node,RREPs_From_Nbr,RREP_Resp_Pct,Pct_Of_All_RREPs,Hop_Cnt_Over_1_Cnt,Hop_Cnt_Over_1_Pct,High_Dest_Seq_Num_Inc_Cnt,High_Dest_Seq_Num_Inc_Pct,Avg_Resp_Dly,Avg_Resp_Dly_Per_Hop,RERRs_From_Nbr,RERRs_From_Nbr_Pct,Pct_of_All_Nbrs,RREP_To_Nbrs_Ratio,Black_Hole_Node
0,1,10.1.1.0,10.1.1.45,44,25,0,181,0,0,True,0,True,621,0,0.00,0.00,0,NaN,0,NaN,NaN,NaN,0,NaN,2.27,0.00,True
1,2,10.1.1.0,10.1.1.2,44,56,127,181,56,0,True,0,True,621,54,29.83,8.70,41,75.93,0,0.0,3.525608,0.946357,17,31.48,2.27,383.26,False
2,3,10.1.1.0,10.1.1.1,44,151,262,181,149,5,False,0,True,621,69,38.12,11.11,31,44.93,0,0.0,1.826252,0.649025,44,63.77,2.27,489.43,False
3,4,10.1.1.0,10.1.1.3,44,42,158,181,63,3,False,0,True,621,67,37.02,10.79,22,32.84,0,0.0,1.357565,0.553015,28,41.79,2.27,475.33,False
4,5,10.1.1.0,10.1.1.31,44,24,49,181,24,0,True,0,True,621,15,8.29,2.42,8,53.33,0,0.0,0.502173,0.199722,10,66.67,2.27,106.61,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,270,10.1.1.5,10.1.1.47,43,5,72,136,32,0,True,0,True,451,36,26.47,7.98,14,38.89,0,0.0,2.003914,0.815869,4,11.11,2.33,342.49,False
270,271,10.1.1.5,10.1.1.17,43,6,106,136,41,22,False,0,True,451,38,27.94,8.43,9,23.68,0,0.0,2.249281,0.893396,27,71.05,2.33,361.80,False
271,272,10.1.1.5,10.1.1.22,43,16,35,136,26,0,True,0,True,451,3,2.21,0.67,2,66.67,0,0.0,2.002772,0.401178,6,200.00,2.33,28.76,False
272,273,10.1.1.5,10.1.1.31,43,7,44,136,18,0,True,0,True,451,15,11.03,3.33,14,93.33,0,0.0,3.086658,0.515923,11,73.33,2.33,142.92,False


In [27]:
# Store the final dataset as a .csv file
file_name = f'dataset_merged.csv'
dir_path = directory_path + "\\"
file_path = dir_path + file_name
dataset_merged.to_csv(file_path)

#### Below here is test code